In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
# create a spark session for youtube data
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Property used to format output tables better
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark

In [4]:
from google.colab import drive
drive.mount('./gdrive')

Mounted at ./gdrive


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!ls "/content/gdrive/MyDrive/pySpark"

ls: cannot access '/content/gdrive/MyDrive/pySpark': Transport endpoint is not connected


In [7]:
!ls "/content/drive/MyDrive/pySpark"

Global_YouTube_Statistics.csv


In [8]:
!cp /content/drive/MyDrive/Nikesh_Web_Clawer /content -r

In [22]:
# Load the csv into the dataframe
youtube_df = spark.read.csv("/content/drive/MyDrive/pySpark/Global_YouTube_Statistics.csv", header=True,  inferSchema=True)
youtube_df

rank,Youtuber,subscribers,video views,category,Title,uploads,Country,Abbreviation,channel_type,video_views_rank,country_rank,channel_type_rank,video_views_for_the_last_30_days,lowest_monthly_earnings,highest_monthly_earnings,lowest_yearly_earnings,highest_yearly_earnings,subscribers_for_last_30_days,created_year,created_month,created_date,Gross tertiary education enrollment (%),Population,Unemployment rate,Urban_population,Latitude,Longitude
1,T-Series,245000000,2.28E11,Music,T-Series,20082,India,IN,Music,1,1,1,2258000000,564600.0,9000000.0,6800000.0,1.084E8,2000000,2006,Mar,13,28.1,1366417754,5.36,471031528,20.593684,78.96288
2,YouTube Movies,170000000,0.0,Film & Animation,youtubemovies,1,United States,US,Games,4055159,7670,7423,12,0.0,0.05,0.04,0.58,nan,2006,Mar,5,88.2,328239523,14.7,270663028,37.09024,-95.712891
3,MrBeast,166000000,2.836884187E10,Entertainment,MrBeast,741,United States,US,Entertainment,48,1,1,1348000000,337000.0,5400000.0,4000000.0,6.47E7,8000000,2012,Feb,20,88.2,328239523,14.7,270663028,37.09024,-95.712891
4,Cocomelon - Nurse...,162000000,1.64E11,Education,Cocomelon - Nurse...,966,United States,US,Education,2,2,1,1975000000,493800.0,7900000.0,5900000.0,9.48E7,1000000,2006,Sep,1,88.2,328239523,14.7,270663028,37.09024,-95.712891
5,SET India,159000000,1.48E11,Shows,SET India,116536,India,IN,Entertainment,3,2,2,1824000000,455900.0,7300000.0,5500000.0,8.75E7,1000000,2006,Sep,20,28.1,1366417754,5.36,471031528,20.593684,78.96288
6,Music,119000000,0.0,nan,Music,0,nan,nan,Music,4057944,nan,nan,nan,0.0,0.0,0.0,0.0,nan,2013,Sep,24,nan,nan,nan,nan,nan,nan
7,��� Kids Diana Show,112000000,9.3247040539E10,People & Blogs,��� Kids Diana Show,1111,United States,US,Entertainment,5,3,3,731674000,182900.0,2900000.0,2200000.0,3.51E7,nan,2015,May,12,88.2,328239523,14.7,270663028,37.09024,-95.712891
8,PewDiePie,111000000,2.9058044447E10,Gaming,PewDiePie,4716,Japan,JP,Entertainment,44,1,4,39184000,9800.0,156700.0,117600.0,1900000.0,nan,2010,Apr,29,63.2,126226568,2.29,115782416,36.204824,138.252924
9,Like Nastya,106000000,9.0479060027E10,People & Blogs,Like Nastya Vlog,493,Russia,RU,People,630,5,25,48947000,12200.0,195800.0,146800.0,2300000.0,100000,2016,Jan,14,81.9,144373535,4.59,107683889,61.52401,105.318756
10,Vlad and Niki,98900000,7.7180169894E10,Entertainment,Vlad and Niki,574,United States,US,Entertainment,8,5,6,580574000,145100.0,2300000.0,1700000.0,2.79E7,600000,2018,Apr,23,88.2,328239523,14.7,270663028,37.09024,-95.712891


In [23]:
youtube_df.printSchema()

root
 |-- rank: integer (nullable = true)
 |-- Youtuber: string (nullable = true)
 |-- subscribers: integer (nullable = true)
 |-- video views: double (nullable = true)
 |-- category: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- uploads: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Abbreviation: string (nullable = true)
 |-- channel_type: string (nullable = true)
 |-- video_views_rank: integer (nullable = true)
 |-- country_rank: string (nullable = true)
 |-- channel_type_rank: string (nullable = true)
 |-- video_views_for_the_last_30_days: string (nullable = true)
 |-- lowest_monthly_earnings: double (nullable = true)
 |-- highest_monthly_earnings: double (nullable = true)
 |-- lowest_yearly_earnings: double (nullable = true)
 |-- highest_yearly_earnings: double (nullable = true)
 |-- subscribers_for_last_30_days: string (nullable = true)
 |-- created_year: string (nullable = true)
 |-- created_month: string (nullable = true)
 |-- created

In [26]:
# data cleaning
# fillna() is used to replace null value with any 0 value
youtube_df.fillna(value=0).show()


+----+--------------------+-----------+---------------+----------------+--------------------+-------+--------------+------------+-------------+----------------+------------+-----------------+--------------------------------+-----------------------+------------------------+----------------------+-----------------------+----------------------------+------------+-------------+------------+---------------------------------------+----------+-----------------+----------------+---------+-----------+
|rank|            Youtuber|subscribers|    video views|        category|               Title|uploads|       Country|Abbreviation| channel_type|video_views_rank|country_rank|channel_type_rank|video_views_for_the_last_30_days|lowest_monthly_earnings|highest_monthly_earnings|lowest_yearly_earnings|highest_yearly_earnings|subscribers_for_last_30_days|created_year|created_month|created_date|Gross tertiary education enrollment (%)|Population|Unemployment rate|Urban_population| Latitude|  Longitude|
+---

In [28]:
# changing the title from rank to Ranking
youtube_df = youtube_df.withColumnRenamed("rank", "Ranking")
youtube_df.show()

+-------+--------------------+-----------+---------------+----------------+--------------------+-------+--------------+------------+-------------+----------------+------------+-----------------+--------------------------------+-----------------------+------------------------+----------------------+-----------------------+----------------------------+------------+-------------+------------+---------------------------------------+----------+-----------------+----------------+---------+-----------+
|Ranking|            Youtuber|subscribers|    video views|        category|               Title|uploads|       Country|Abbreviation| channel_type|video_views_rank|country_rank|channel_type_rank|video_views_for_the_last_30_days|lowest_monthly_earnings|highest_monthly_earnings|lowest_yearly_earnings|highest_yearly_earnings|subscribers_for_last_30_days|created_year|created_month|created_date|Gross tertiary education enrollment (%)|Population|Unemployment rate|Urban_population| Latitude|  Longitude

In [51]:
# Dropping all rows with null values on any columns
total_attritions = youtube_df.count()
non_na_attritions = youtube_df.na.drop("all").count()
print("Total rows = ", total_attritions)
print("Total Non-NA rows = ", non_na_attritions)
print("Total NA rows = ", total_attritions - non_na_attritions)
origData = youtube_df.dropna(how='all')

Total rows =  995
Total Non-NA rows =  995
Total NA rows =  0


In [52]:
# Dropping duplicate records if any
total_attritions = youtube_df.count()
unique_attritions = youtube_df.dropDuplicates().count()
print("Total rows = ", total_attritions)
print("Total unique rows = ", unique_attritions)
print("Total duplicate rows = ", total_attritions - unique_attritions)

Total rows =  995
Total unique rows =  995
Total duplicate rows =  0


In [29]:
youtube_df.limit(5)

Ranking,Youtuber,subscribers,video views,category,Title,uploads,Country,Abbreviation,channel_type,video_views_rank,country_rank,channel_type_rank,video_views_for_the_last_30_days,lowest_monthly_earnings,highest_monthly_earnings,lowest_yearly_earnings,highest_yearly_earnings,subscribers_for_last_30_days,created_year,created_month,created_date,Gross tertiary education enrollment (%),Population,Unemployment rate,Urban_population,Latitude,Longitude
1,T-Series,245000000,2.28E11,Music,T-Series,20082,India,IN,Music,1,1,1,2258000000,564600.0,9000000.0,6800000.0,1.084E8,2000000,2006,Mar,13,28.1,1366417754,5.36,471031528,20.593684,78.96288
2,YouTube Movies,170000000,0.0,Film & Animation,youtubemovies,1,United States,US,Games,4055159,7670,7423,12,0.0,0.05,0.04,0.58,nan,2006,Mar,5,88.2,328239523,14.7,270663028,37.09024,-95.712891
3,MrBeast,166000000,2.836884187E10,Entertainment,MrBeast,741,United States,US,Entertainment,48,1,1,1348000000,337000.0,5400000.0,4000000.0,6.47E7,8000000,2012,Feb,20,88.2,328239523,14.7,270663028,37.09024,-95.712891
4,Cocomelon - Nurse...,162000000,1.64E11,Education,Cocomelon - Nurse...,966,United States,US,Education,2,2,1,1975000000,493800.0,7900000.0,5900000.0,9.48E7,1000000,2006,Sep,1,88.2,328239523,14.7,270663028,37.09024,-95.712891
5,SET India,159000000,1.48E11,Shows,SET India,116536,India,IN,Entertainment,3,2,2,1824000000,455900.0,7300000.0,5500000.0,8.75E7,1000000,2006,Sep,20,28.1,1366417754,5.36,471031528,20.593684,78.96288


In [31]:
youtube_df.agg({'lowest_yearly_earnings':'avg'})

avg(lowest_yearly_earnings)
442257.3925326631


In [53]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf

def round_float_down(x):
  return int(x)

round_float_down_udf = udf(round_float_down, IntegerType())

youtube_df.select('Youtuber', 'category', 'Title',
   round_float_down_udf('lowest_yearly_earnings')
  .alias('Lowest early earning')).limit(5)

Youtuber,category,Title,Lowest early earning
T-Series,Music,T-Series,6800000
YouTube Movies,Film & Animation,youtubemovies,0
MrBeast,Entertainment,MrBeast,4000000
Cocomelon - Nurse...,Education,Cocomelon - Nurse...,5900000
SET India,Shows,SET India,5500000


In [33]:
youtube_df.createOrReplaceTempView("Youtube")

In [34]:
spark.sql('select * from Youtube')

Ranking,Youtuber,subscribers,video views,category,Title,uploads,Country,Abbreviation,channel_type,video_views_rank,country_rank,channel_type_rank,video_views_for_the_last_30_days,lowest_monthly_earnings,highest_monthly_earnings,lowest_yearly_earnings,highest_yearly_earnings,subscribers_for_last_30_days,created_year,created_month,created_date,Gross tertiary education enrollment (%),Population,Unemployment rate,Urban_population,Latitude,Longitude
1,T-Series,245000000,2.28E11,Music,T-Series,20082,India,IN,Music,1,1,1,2258000000,564600.0,9000000.0,6800000.0,1.084E8,2000000,2006,Mar,13,28.1,1366417754,5.36,471031528,20.593684,78.96288
2,YouTube Movies,170000000,0.0,Film & Animation,youtubemovies,1,United States,US,Games,4055159,7670,7423,12,0.0,0.05,0.04,0.58,nan,2006,Mar,5,88.2,328239523,14.7,270663028,37.09024,-95.712891
3,MrBeast,166000000,2.836884187E10,Entertainment,MrBeast,741,United States,US,Entertainment,48,1,1,1348000000,337000.0,5400000.0,4000000.0,6.47E7,8000000,2012,Feb,20,88.2,328239523,14.7,270663028,37.09024,-95.712891
4,Cocomelon - Nurse...,162000000,1.64E11,Education,Cocomelon - Nurse...,966,United States,US,Education,2,2,1,1975000000,493800.0,7900000.0,5900000.0,9.48E7,1000000,2006,Sep,1,88.2,328239523,14.7,270663028,37.09024,-95.712891
5,SET India,159000000,1.48E11,Shows,SET India,116536,India,IN,Entertainment,3,2,2,1824000000,455900.0,7300000.0,5500000.0,8.75E7,1000000,2006,Sep,20,28.1,1366417754,5.36,471031528,20.593684,78.96288
6,Music,119000000,0.0,nan,Music,0,nan,nan,Music,4057944,nan,nan,nan,0.0,0.0,0.0,0.0,nan,2013,Sep,24,nan,nan,nan,nan,nan,nan
7,��� Kids Diana Show,112000000,9.3247040539E10,People & Blogs,��� Kids Diana Show,1111,United States,US,Entertainment,5,3,3,731674000,182900.0,2900000.0,2200000.0,3.51E7,nan,2015,May,12,88.2,328239523,14.7,270663028,37.09024,-95.712891
8,PewDiePie,111000000,2.9058044447E10,Gaming,PewDiePie,4716,Japan,JP,Entertainment,44,1,4,39184000,9800.0,156700.0,117600.0,1900000.0,nan,2010,Apr,29,63.2,126226568,2.29,115782416,36.204824,138.252924
9,Like Nastya,106000000,9.0479060027E10,People & Blogs,Like Nastya Vlog,493,Russia,RU,People,630,5,25,48947000,12200.0,195800.0,146800.0,2300000.0,100000,2016,Jan,14,81.9,144373535,4.59,107683889,61.52401,105.318756
10,Vlad and Niki,98900000,7.7180169894E10,Entertainment,Vlad and Niki,574,United States,US,Entertainment,8,5,6,580574000,145100.0,2300000.0,1700000.0,2.79E7,600000,2018,Apr,23,88.2,328239523,14.7,270663028,37.09024,-95.712891


In [49]:
query = """
  SELECT Youtuber, subscribers, country
  FROM Youtube
  WHERE Country == "India"
  ORDER BY subscribers DESC
  LIMIT 10;
"""

spark.sql(query).show()

+--------------------+-----------+-------+
|            Youtuber|subscribers|country|
+--------------------+-----------+-------+
|            T-Series|  245000000|  India|
|           SET India|  159000000|  India|
|   Zee Music Company|   96700000|  India|
|            Sony SAB|   83000000|  India|
|              Zee TV|   70500000|  India|
|ChuChu TV Nursery...|   65900000|  India|
|Shemaroo Filmi Gaane|   65600000|  India|
|           Colors TV|   64600000|  India|
|T-Series Bhakti S...|   61000000|  India|
|       Tips Official|   59300000|  India|
+--------------------+-----------+-------+

